In [ ]:
from joblib import logger
import pandas as pd
import transforms_distributed as tfd
# import tc_distributed as tcd
import tc_distributed_pro as tcdp
from dask.distributed import LocalCluster, Client, progress
from dask_jobqueue import HTCondorCluster
import logging, coloredlogs
import sys, os
import copy
import pathlib
import warnings

dataset = 'data/AP_Omentum_Ovary.csv'
# dataset = 'data/R/Openml_6071.csv'
datapath = 'result/' + dataset[5:-4]
art_new, start_idx = 'C', 5
if dataset[5] == 'R':
    art_new, start_idx = 'R', 7

logging.basicConfig(level=logging.ERROR)
# logging.basicConfig(level=logging.ERROR, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logger_new = logging.getLogger(__name__)
# output_file_handler = logging.FileHandler("log/output_" + dataset[start_idx:-3] + 'log')
output_file_handler = logging.FileHandler("log/R/output_" + dataset[start_idx:-3] + 'log')
output_file_handler.setFormatter(formatter)
logger_new.addHandler(output_file_handler)
coloredlogs.install(level = 'DEBUG')
pathlib.Path(datapath).mkdir(parents=True, exist_ok=True)
warnings.filterwarnings("ignore")

## Generate Features

In [ ]:
dat = tcdp.load_data(dataset, logger=logger_new, art=art_new)
group = dat.drop(dat.columns[-1], axis=1, inplace=False)
res = tcdp.eval(dat.iloc[:, :-1], dat.iloc[:, -1], art=art_new)
res, v = tcdp.compute(res)[0]
logger_new.info("For original dataset %s, mean score is %f, standard variance is %f" %(str(dataset[start_idx:-4]), res, v))
for num in range(10, 101, 10):
# num = 50
    dat_sel = tcdp.bestFeatures(dat, num, art=art_new)
    cur_dat = dat_sel
    cur_dat[dat.columns[-1]] = dat.iloc[:, -1]
    group = cur_dat.drop(cur_dat.columns[-1], axis=1, inplace=False)
    res = tcdp.calculateFitness(dat, group, art=art_new)
    res, v = tcdp.compute(res)[0]
    logger_new.info("For dataset %s which selects %d features from original, its mean score is %f, standard variance is %f" %(str(dataset[start_idx:-4]), num, res, v))

In [ ]:
repeat = 4
una_oprs = tfd.unary_operators
bina_oprs = tfd.binary_operators
weights = [repeat] * (len(una_oprs) + len(bina_oprs))
# import data
dat = tcdp.load_data(dataset, logger=logger_new, art=art_new)
# the number of next generation's features not over inflation*num_of_curr_gen to form an up-side-down pyramid structure
# magic_number: inflation, cur_limit, total_limit, num_best_features
inflation = 10
cur_size = dat.shape[1]-1
# total_limit can be much larger, which denpends on the device
cur_limit = 2000
total_limit = 1000
cur_dat = dat
prev_gen = None
# only for the dataset with too many original features
dat_sel = None

if dat.shape[1] > 10000:
    # for AP_Omentum_Ovary
    group = dat.drop(dat.columns[-1], axis=1, inplace=False)
    res = tcdp.eval(dat.iloc[:, :-1], dat.iloc[:, -1], art=art_new)
    res, v = tcdp.compute(res)[0]
    logger_new.info("For original dataset %s, mean score is %f, standard variance is %f" %(str(dataset[start_idx:-4]), res, v))
    num = 50
    dat_sel = tcdp.bestFeatures(dat, num, art=art_new)
    cur_dat = dat_sel
    cur_dat[dat.columns[-1]] = dat.iloc[:, -1]
    group = cur_dat.drop(cur_dat.columns[-1], axis=1, inplace=False)
    res = tcdp.calculateFitness(dat, group, art=art_new)
    res, v = tcdp.compute(res)[0]
    logger_new.info("For dataset %s which selects %d features from original, its mean score is %f, standard variance is %f" 
                    %(str(dataset[start_idx:-4]), num, res, v))

# generate and store candidate features
for i in range(repeat):
    cur_dat, gen = tcdp.updateDat(cur_dat, prev_gen=prev_gen, oprs_weights=weights, art=art_new, logger=logger_new)
    cur_gen, cur_size = tcdp.constrainFeaturesNum(cur_dat, min(inflation*cur_size, cur_limit), art=art_new, logger=logger_new)
    logger_new.debug('The number of features in cur-gen drops from %d to %d' %(cur_dat.shape[1]-1, cur_size))

    prev_gen = pd.concat([prev_gen, gen], axis=1)
    prev_gen[dat.columns[-1]] = dat.iloc[:, -1]
    prev_gen, prev_size = tcdp.constrainFeaturesNum(prev_gen, total_limit, art=art_new, logger=logger_new)
    logger_new.debug('The number of features in prev-gen drops from %d to %d' %(prev_gen.shape[1], prev_size))

    if dat_sel is not None:
        cur_dat = tcdp.addInitalFeatures(cur_gen, prev_gen, dat_sel, logger=logger_new)
    else:
        cur_dat = tcdp.addInitalFeatures(cur_gen, prev_gen, dat, logger=logger_new)
    cur_dat.to_csv(datapath + '/gen' + str(i+1) + '.csv')

total_dat = pd.concat([prev_gen, cur_dat], axis=1)
total_dat.to_csv(datapath + '/total.csv')
# drop the columns with high correlations
# total_dat = tcdp.dropHighCorrelation(total_dat, logger=logger_new)
# total_dat[dat.columns[-1]] = dat.iloc[:, -1]

## Select features from total and generate final

In [ ]:
# dat = tcdp.load_data(dataset, art=art_new)
gen1 = pd.read_csv(datapath + '/gen1.csv')
gen1.drop(gen1.columns[0], axis=1, inplace=True)
gen1.drop(gen1.columns[-1], axis=1, inplace=True)
gen2 = pd.read_csv(datapath + '/gen2.csv')
gen2.drop(gen2.columns[0], axis=1, inplace=True)
gen2.drop(gen2.columns[-1], axis=1, inplace=True)
gen3 = pd.read_csv(datapath + '/gen3.csv')
gen3.drop(gen3.columns[0], axis=1, inplace=True)
# gen3.drop(gen3.columns[-1], axis=1, inplace=True)
# gen4 = pd.read_csv(datapath + '/gen4.csv')
# gen4.drop(gen4.columns[0], axis=1, inplace=True)
total_dat = pd.concat([gen1, gen2, gen3], axis=1)
total_dat.to_csv(datapath + '/total' + '.csv')

In [ ]:
dat = tcdp.load_data(dataset, art=art_new)
# dat = pd.read_csv(dataset)
print(dat.shape[1])
res = tcdp.eval(dat.iloc[:, :-1],dat.iloc[:, -1], art=art_new)
res, v = tcdp.compute(res)[0]
print("For dataset %s, mean score is %f, standard variance is %f" %(str(dataset[start_idx:-4]), res, v))

total_dat = pd.read_csv(datapath + '/total.csv')
total_dat.drop(total_dat.columns[0], axis=1, inplace=True)

In [ ]:
best_features_cands = tcdp.bestFeatures(total_dat, 30, art=art_new)
init_fitness, cur_fitness = tcdp.scoreCompare(dat, best_features_cands, art=art_new, logger=logger_new)
cur_fitness

In [ ]:
for i in range(1, 10):
    i /= 10
    num_best_features = round(i * total_dat.shape[1])
    best_features_cands = tcdp.bestFeatures(total_dat, num_best_features, art=art_new)
    init_fitness, cur_fitness = tcdp.scoreCompare(dat, best_features_cands, art=art_new, logger=logger_new)
    logger_new.info("Finish selecting best %d features (coefficient: %f) in the first round coarsely" %(num_best_features, i))
    increase = (cur_fitness - init_fitness) / init_fitness
    logger_new.debug("After the first round, the fitness is %f compared with the initial one, the fitness increased by %s" %(cur_fitness, str(increase)))


In [ ]:
# grid search for best combination of coef1 and coef2
max_inc, coef1, coef2 = 0, 0, 0
dat_sel = None
for i in range(1, 10):
    i /= 10
    num_best_features = round(i * total_dat.shape[1])
    best_features_cands = tcdp.bestFeatures(total_dat, num_best_features, art=art_new)
    last = best_features_cands.shape[1]
    for j in range(10, 0, -1):
        j /= 10
        if dat_sel is not None:
            size_limit = max(3, round(j * dat_sel.shape[1]))
        else:
            size_limit = max(3, round(j * dat.shape[1]))
        if size_limit > last:
            continue
        best_features = pd.DataFrame(best_features_cands)
        best_features[total_dat.columns[-1]] = total_dat.iloc[:, -1]
        while best_features.shape[1] > size_limit:
            best_features = tcdp.featureSelection(best_features, art=art_new)
        last = best_features.shape[1]
        best_features.drop(best_features.columns[-1], axis=1, inplace=True)
        init_fitness, cur_fitness = tcdp.scoreCompare(dat, best_features, art=art_new)
        increase = (cur_fitness - init_fitness) / init_fitness
        logger_new.debug("with coef1=%f, coef2=%f, the increasing is %f" %(i, j, increase))
        if increase > max_inc:
            max_inc, coef1, coef2 = increase, i, j
logger_new.debug("After grid search of best coeffs, with coef1: %f coef2: %f, the increase maximized by %f" %(coef1, coef2, max_inc))

In [ ]:
# Select best K features from the total candidate features
dat_sel = None
coef1 = 0.9
num_best_features = round(coef1 * total_dat.shape[1])
best_features_cands = tcdp.bestFeatures(total_dat, num_best_features, art=art_new, logger=logger_new)
init_fitness, cur_fitness = tcdp.scoreCompare(dat, best_features_cands, art=art_new, logger=logger_new)
logger_new.info("Finish selecting best %d features (coefficient: %f) in the first round coarsely" %(num_best_features, coef1))
increase = (cur_fitness - init_fitness) / init_fitness
logger_new.debug("After the first round, the fitness is %f compared with the initial one, the fitness increased by %s" %(cur_fitness, str(increase)))

# reduce the features number on the basis of a higher score compared to the initial one
coef2 = 0.9
if dat_sel is not None:
    size_limit = max(3, round(coef2 * dat_sel.shape[1]))
else:
    size_limit = max(3, round(coef2 * dat.shape[1]))
logger_new.info("The limit of the final gen's size is %d with coeffiecient %f" %(size_limit, coef2))
best_features = pd.DataFrame(best_features_cands)
best_features[total_dat.columns[-1]] = total_dat.iloc[:, -1]
print(size_limit)
while best_features.shape[1] > size_limit:
    best_features = tcdp.featureSelection(best_features, art=art_new, logger=logger_new)
    print("best_features: %d" %(best_features.shape[1]))
best_features.drop(best_features.columns[-1], axis=1, inplace=True)
logger_new.debug("Reduce the size of the final selected features to %d by featureSelection at last" %(best_features.shape[1]))

init_fitness, cur_fitness = tcdp.scoreCompare(dat, best_features, art=art_new, logger=logger_new)
increase = (cur_fitness - init_fitness) / init_fitness
logger_new.debug("At last, compared with the initial one, the fitness increased by %s" %(str(increase)))
best_features.to_csv(datapath + '/final' + '.csv')

In [ ]:
# only random forest in feature selection
coef2 = 1
size_limit = max(3, round(coef2 * dat.shape[1]))
best_features = total_dat
while best_features.shape[1] > size_limit:
    best_features = tcdp.featureSelection(best_features, art=art_new, logger=logger_new)
    print("best_features: %d" %(best_features.shape[1]))
best_features.drop(best_features.columns[-1], axis=1, inplace=True)
logger_new.debug("Only after fs with random forest, reduce the size of all candidates to %d" %(best_features.shape[1]))

init_fitness, cur_fitness = tcdp.scoreCompare(dat, best_features, art=art_new, logger=logger_new)
increase = (cur_fitness - init_fitness) / init_fitness
logger_new.debug("With only one fs step, compared with the initial one, the fitness increased by %s" %(str(increase)))

In [ ]:
# only selectKBest in fs


## The First Version

In [ ]:
dataset = 'data/AP_Omentum_Ovary.csv'
datapath = 'result/' + dataset[5:-4]

logging.basicConfig(level=logging.ERROR)
# logging.basicConfig(level=logging.ERROR, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logger_new = logging.getLogger(__name__)

output_file_handler = logging.FileHandler("log/output2_" + dataset[5:-3] + 'log')
output_file_handler.setFormatter(formatter)
logger_new.addHandler(output_file_handler)
coloredlogs.install(level = 'DEBUG')

# stdout_handler = logging.StreamHandler(sys.stdout)
# stdout_handler.setFormatter(formatter)
# logger_new.addHandler(stdout_handler)

# cluster = LocalCluster(n_workers=4, threads_per_worker=2, memory_limit='2GB')
# client = Client(cluster)

In [ ]:
una_oprs = tfd.unary_operators
bina_oprs = tfd.binary_operators

pathlib.Path(datapath).mkdir(parents=True, exist_ok=True)
repeat = 1
weights = [repeat] * (len(una_oprs) + len(bina_oprs))
# load data
dat = tcdp.load_data(dataset, logger=logger_new, art='C')
cur_dat = dat
prev_gen = None
total_limit = 400
prev_size = 0

for i in range(repeat):
    cur_dat.to_csv(datapath + '/gen' + str(i) + '.csv')
    cur_dat = tcdp.update_dat(cur_dat, prev_gen=prev_gen, oprs_weights=weights, art='C', logger=logger_new)
    cur_gen, prev_size = tcdp.pyramid_cur(cur_dat, prev_size, art='C', logger=logger_new)

    prev_gen = pd.concat([prev_gen, cur_gen], axis=1)
    # total_limit can be much larger, which denpends on the device
    prev_gen[dat.columns[-1]] = dat.iloc[:, -1]
    while prev_gen.shape[1] > total_limit:
        logger.info('The number of prev_gen columns %d exceed prev_limit %d, columns selection first' % (prev_gen.shape[1], prev_limit))
        prev_gen = tcdp.feature_selection(prev_gen, art=art, logger=logger)
    prev_gen.drop(prev_gen.columns[-1], axis=1, inplace=True)

    # add in intial features
    if (i % 2):
        cur_dat = pd.concat([cur_gen, dat], axis=1)
    else :
        cur_gen[dat.columns[-1]] = dat.iloc[:, -1]
        cur_dat = cur_gen


In [ ]:
# delete the first column of the data after read_csv
cur_dat = pd.read_csv('result/curr_php0iVrYT.csv')
cur_dat.drop(cur_dat.columns[0], axis=1, inplace=True)
cur_gen = pd.DataFrame(cur_dat.iloc[:, :-1], columns=cur_dat.columns[:-1])
# print(cur_gen)
prev_dat = pd.read_csv('result/prev_php0iVrYT.csv')
prev_gen = prev_dat.drop(prev_dat.columns[0], axis=1, inplace=False)
# prev_gen

In [ ]:
for i in range(3):
    cur_dat, prev_gen = tcd.update_dat(cur_dat, prev_gen=prev_gen, logger=logger_new)
    cur_dat.to_csv('result/curr_' + dataset[5:])
    prev_gen.to_csv('result/prev_' + dataset[5:])
logger_new.warning('+Successfully finish all steps')